<a href="https://colab.research.google.com/github/Singular-Brain/DeepBioLCNet/blob/main/CartPole_FCSNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook setups

In [1]:
!pip install -q git+https://github.com/Singular-Brain/DeepBioLCNet

     |████████████████████████████████| 831.4 MB 2.7 kB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 120 kB 85.6 MB/s 
     |████████████████████████████████| 74 kB 4.0 MB/s 
     |████████████████████████████████| 296 kB 88.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
# !git clone https://github.com/Singular-Brain/DeepBioLCNet


In [3]:
### gym and colab compatibility

# !apt-get install -y xvfb python-opengl > /dev/null 2>&1
# !pip install gym pyvirtualdisplay > /dev/null 2>&1

import gym
import numpy as np
import matplotlib.pyplot as plt
# from IPython import display as ipythondisplay

# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(400, 300))
# display.start()

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from itertools import count
from matplotlib.axes import Axes
from matplotlib.image import AxesImage
from torch.nn.modules.utils import _pair
from matplotlib.collections import PathCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
from typing import Tuple, List, Optional, Sized, Dict, Union
import math
import random
from torchvision.transforms.functional import crop
# from ..utils import reshape_locally_connected_weights, reshape_locally_connected_weights_meh, reshape_conv2d_weights

import gym
import tkinter

from PIL import Image
from collections import namedtuple

In [5]:
from bindsnet.network.nodes import Nodes
import os
### import torch
import numpy as np
import copy
import math
import matplotlib.pyplot as plt
import collections
from torchvision import transforms
from tqdm.notebook import tqdm

from sklearn.metrics import confusion_matrix
import seaborn as sn
import torch.nn.functional as fn

from abc import ABC, abstractmethod
from typing import Union, Tuple, Optional, Sequence
from torch.nn.modules.utils import _pair

from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from bindsnet.network import Network
from bindsnet.network.nodes import Input, LIFNodes, AdaptiveLIFNodes, IFNodes
from bindsnet.network.topology import Connection, MaxPool2dLocalConnection
from bindsnet.network.topology import LocalConnection, LocalConnectionOrig
from bindsnet.network.monitors import Monitor, AbstractMonitor, TensorBoardMonitor
from bindsnet.learning import PostPre, MSTDP, MSTDPET, WeightDependentPostPre, Hebbian
from bindsnet.learning.reward import DynamicDopamineInjection, DopaminergicRPE, AbstractReward
from bindsnet.analysis.plotting import plot_locally_connected_weights,plot_locally_connected_weights_meh,plot_spikes,\
plot_LC_timepoint_spikes,plot_locally_connected_weights_meh2,plot_convergence_and_histogram,plot_locally_connected_weights_meh3
from bindsnet.analysis.visualization import plot_weights_movie, plot_spike_trains_for_example,summary, plot_voltage
from bindsnet.utils import reshape_locally_connected_weights, reshape_locally_connected_weights_meh, reshape_conv2d_weights

In [71]:
class RLTasks(AbstractReward):
    # language=rst
    """
    Computes the reward for a given RL task in the current state
    """
    def __init__(
        self,
        env_id,
        **kwargs,
    ) -> None:

        super().__init__()

        if env_id == "CartPole-v0":
            self.compute = self._cartPole_compute
        elif env_id == "MountainCar-v0-v0":
            self.compute = self._mountainCar_compute
        elif env_id == "BreakoutDeterministic-v4":
            self.compute = self._breakout_compute
        else:
            raise NotImplementedError(
                "This rl environment is not currently supported."
            )
    
    def _cartPole_compute(self, **kwargs):
        success = kwargs['success']
        failure = kwargs['failure']
        if kwargs['classic_reward']:
            if failure:
                return torch.tensor([0.])
            else:
                return torch.tensor([1.])

        env = kwargs['env']
        state = env.state
        success = kwargs['success']
        failure = kwargs['failure']
        x, x_dot, theta, theta_dot = state
        # r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        # r2 = ((env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5)#*2
        #reward = r1 + r2
        reward = ((env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians) #- 0.5*abs(theta_dot)
        if success:
            reward += 5
        elif failure:
            reward -= 5
        reward = torch.tensor([reward])
        return reward

    def _mountainCar_compute(self):
        pass

    def _breakout_compute(self):
        pass
        
    def update(self):
        pass

    def online_compute(self,):
        pass

## Sets up Gpu use and manual seed


In [72]:
if torch.cuda.is_available():
    device =  torch.device("cuda")
    gpu = True
else:
    device =  torch.device("cpu")
    gpu = False

def manual_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


SEED = 2045 # The Singularity is Near!
manual_seed(SEED)
WANDB = False

torch.set_num_threads(os.cpu_count() - 1)
print("Running on Device = ", device)


Running on Device =  cuda


# Custom Monitors

## Reward Monitor

In [73]:
class RewardMonitor(AbstractMonitor):
    # language=rst
    """
    Records state variables of interest.
    """

    def __init__(
        self,
        time: None,
        batch_size: int = 1,
        device: str = "cpu",
    ):
        # language=rst
        """
        Constructs a ``Monitor`` object.

        :param obj: An object to record state variables from during network simulation.
        :param state_vars: Iterable of strings indicating names of state variables to record.
        :param time: If not ``None``, pre-allocate memory for state variable recording.
        :param device: Allow the monitor to be on different device separate from Network device
        """
        super().__init__()

        self.time = time
        self.batch_size = batch_size
        self.device = device

        # if time is not specified the monitor variable accumulate the logs
        if self.time is None:
            self.device = "cpu"

        self.recording = []
        self.reset_state_variables()

    def get(self,) -> torch.Tensor:
        # language=rst
        """
        Return recording to user.

        :return: Tensor of shape ``[time, n_1, ..., n_k]``, where ``[n_1, ..., n_k]`` is the shape of the recorded state
        variable.
        Note, if time == `None`, get return the logs and empty the monitor variable

        """
        # return_logs = torch.as_tensor(self.recording)
        # if self.time is None:
        #     self.recording = []
        return self.recording

    def record(self, **kwargs) -> None:
        # language=rst
        """
        Appends the current value of the recorded state variables to the recording.
        """
        if "reward" in kwargs:
            self.recording.append(kwargs["reward"])
        # remove the oldest element (first in the list)
        # if self.time is not None:
        #     self.recording.pop(0)

    def reset_state_variables(self) -> None:
        # language=rst
        """
        Resets recordings to empty ``List``s.
        """
        self.recording = []


## Plot Eligibility trace

In [74]:
class PlotET(AbstractMonitor):
    # language=rst
    """
    Records state variables of interest.
    """

    def __init__(
        self,
        i,
        j,
        source,
        target,
        connection,
    ):
        # language=rst
        """
        Constructs a ``Monitor`` object.

        :param obj: An object to record state variables from during network simulation.
        :param state_vars: Iterable of strings indicating names of state variables to record.
        :param time: If not ``None``, pre-allocate memory for state variable recording.
        :param device: Allow the monitor to be on different device separate from Network device
        """
        super().__init__()
        self.i = i
        self.j = j
        self.source = source
        self.target = target
        self.connection = connection

        self.reset_state_variables()

    def get(self,) -> torch.Tensor:
        # language=rst
        """
        Return recording to user.

        :return: Tensor of shape ``[time, n_1, ..., n_k]``, where ``[n_1, ..., n_k]`` is the shape of the recorded state
        variable.
        Note, if time == `None`, get return the logs and empty the monitor variable

        """
        # return_logs = torch.as_tensor(self.recording)
        # if self.time is None:
        #     self.recording = []
        return self.recording

    def record(self, **kwargs) -> None:
        # language=rst
        """
        Appends the current value of the recorded state variables to the recording.
        """
        if hasattr(self.connection.update_rule, 'p_plus'):
            self.recording['spikes_i'].append(self.source.s.ravel()[self.i].item())
            self.recording['spikes_j'].append(self.target.s.ravel()[self.j].item())
            self.recording['p_plus'].append(self.connection.update_rule.p_plus[self.i].item())
            self.recording['p_minus'].append(self.connection.update_rule.p_minus[self.j].item())
            self.recording['eligibility'].append(self.connection.update_rule.eligibility[self.i,self.j].item())
            self.recording['eligibility_trace'].append(self.connection.update_rule.eligibility_trace[self.i,self.j].item())
            self.recording['w'].append(self.connection.w[self.i,self.j].item())

    def plot(self):

        fig, axs  = plt.subplots(7)
        fig.set_size_inches(10, 20)
        for i, (name, p) in enumerate(self.recording.items()):
            axs[i].plot(p[-250:])
            axs[i].set_title(name)
    
        fig.show()

    def reset_state_variables(self) -> None:
        # language=rst
        """
        Resets recordings to empty ``List``s.
        """
        self.recording = {
        'spikes_i': [],
        'spikes_j': [],
        'p_plus':[],
        'p_minus':[],
        'eligibility':[],
        'eligibility_trace':[],
        'w': [],
        }


## Kernel 

In [75]:
class AbstractKernel(ABC):
    def __init__(self, kernel_size):
        """
        Base class for generating image filter kernels such as Gabor, DoG, etc. Each subclass should override :attr:`__call__` function.
        Instantiates a ``Filter Kernel`` object.
        :param window_size : The size of the kernel (int)
        """
        self.window_size = kernel_size

    def __call__(self):
        pass


In [76]:
class DoGKernel(AbstractKernel):
    def __init__(self, kernel_size: Union[int, Tuple[int, int]], sigma1 : float, sigma2 : float):
        """
        Generates DoG filter kernels.
        :param kernel_size: Horizontal and vertical size of DOG kernels.(If pass int, we consider it as a square filter) 
        :param sigma1 : The sigma parameter for the first Gaussian function.
        :param sigma2 : The sigma parameter for the second Gaussian function.
        """
        super(DoGKernel, self).__init__(kernel_size)
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        
    def __call__(self):
        k = self.window_size//2
        x, y = np.mgrid[-k:k+1:1, -k:k+1:1]
        a = 1.0 / (2 * math.pi)
        prod = x*x + y*y
        f1 = (1/(self.sigma1*self.sigma1)) * np.exp(-0.5 * (1/(self.sigma1*self.sigma1)) * (prod))
        f2 = (1/(self.sigma2*self.sigma2)) * np.exp(-0.5 * (1/(self.sigma2*self.sigma2)) * (prod))
        dog = a * (f1-f2)
        dog_mean = np.mean(dog)
        dog = dog - dog_mean
        dog_max = np.max(dog)
        dog = dog / dog_max
        dog_tensor = torch.from_numpy(dog)
        # returns a 2d tensor corresponding to the requested DoG filter
        return dog_tensor.float()

In [77]:
class Filter():
    """
    Applies a filter transform. Each filter contains a sequence of :attr:`FilterKernel` objects.
    The result of each filter kernel will be passed through a given threshold (if not :attr:`None`).
    Args:
        filter_kernels (sequence of FilterKernels): The sequence of filter kernels.
        padding (int, optional): The size of the padding for the convolution of filter kernels. Default: 0
        thresholds (sequence of floats, optional): The threshold for each filter kernel. Default: None
        use_abs (boolean, optional): To compute the absolute value of the outputs or not. Default: False
    .. note::
        The size of the compund filter kernel tensor (stack of individual filter kernels) will be equal to the 
        greatest window size among kernels. All other smaller kernels will be zero-padded with an appropriate 
        amount.
    """
    # filter_kernels must be a list of filter kernels
    # thresholds must be a list of thresholds for each kernel
    def __init__(self, filter_kernels, padding=0, thresholds=None, use_abs=False):
        tensor_list = []
        self.max_window_size = 0
        for kernel in filter_kernels:
            if isinstance(kernel, torch.Tensor):
                tensor_list.append(kernel)
                self.max_window_size = max(self.max_window_size, kernel.size(-1))
            else:
                tensor_list.append(kernel().unsqueeze(0))
                self.max_window_size = max(self.max_window_size, kernel.window_size)
        for i in range(len(tensor_list)):
            p = (self.max_window_size - filter_kernels[i].window_size)//2
            tensor_list[i] = fn.pad(tensor_list[i], (p,p,p,p))

        self.kernels = torch.stack(tensor_list)
        self.number_of_kernels = len(filter_kernels)
        self.padding = padding
        if isinstance(thresholds, list):
            self.thresholds = thresholds.clone().detach()
            self.thresholds.unsqueeze_(0).unsqueeze_(2).unsqueeze_(3)
        else:
            self.thresholds = thresholds
        self.use_abs = use_abs

    # returns a 4d tensor containing the flitered versions of the input image
    # input is a 4d tensor. dim: (minibatch=1, filter_kernels, height, width)
    def __call__(self, input):

        # if input.dim() == 3:
        #     input2 = torch.unsqueeze(input, 0)
        input.unsqueeze_(0)
        output = fn.conv2d(input, self.kernels, padding = self.padding).float()
        if not(self.thresholds is None):
            output = torch.where(output < self.thresholds, torch.tensor(0.0, device=output.device), output)
        if self.use_abs:
            torch.abs_(output)
        return output.squeeze(0)

# Viz

In [78]:
def plot_LC_timepoint_spikes(spikes: torch.Tensor,
    timepoint: int,
    n_filters: int,
    in_chans: int,
    slice_to_plot: int,
    conv_size: Union[int, Tuple[int, int]],
    im: Optional[AxesImage] = None,
    lines: bool = True,
    figsize: Tuple[int, int] = (10, 10),
    cmap: str = "hot_r",
    color: str='r'
    ):
    # language=rst
    """
    """

    n_sqrt = int(np.ceil(np.sqrt(n_filters)))
    sel_slice = spikes[timepoint].view(in_chans, n_filters, conv_size, conv_size).cpu()
    sel_slice = sel_slice[slice_to_plot, ...].view(n_filters, conv_size, conv_size)
    spikes_ = np.zeros((n_sqrt*conv_size, n_sqrt*conv_size))
    filt_counter = 0

    fig, ax = plt.subplots(figsize=figsize)

    for n1 in range(n_sqrt):
        for n2 in range(n_sqrt):
            filter_ = sel_slice[filt_counter, :, :].view(conv_size, conv_size)
            spikes_[n1 * conv_size : (n1 + 1) * conv_size, n2 * conv_size : (n2 + 1) * conv_size] = filter_
            filt_counter += 1
            ax.axhline((n1 + 1) * conv_size, color="g", linestyle="-")
            ax.axvline((n2 + 1) * conv_size, color="g", linestyle="--")
    ax.imshow(spikes_, cmap='Greys')
    return spikes_
    
def plot_FC_response_map(lc: object,
    fc: object,
    ind_neuron_in_group: int,
    label: int,
    n_per_action: int,
    input_channel: int = 0,
    scale_factor: float = 1.0,
    lines: bool = True,
    figsize: Tuple[int, int] = (5, 5),
    cmap: str = "hot_r",
    color: str='r'
    ) -> AxesImage:
    # language=rst
    """
    Plot a connection weight matrix of a :code:`Connection` with `locally connected
    structure <http://yann.lecun.com/exdb/publis/pdf/gregor-nips-11.pdf>_.
    :param lc: LC connection object of LCNet
    :param fc: FC connection object of LCNet
    :param input_channel: indicates weights which connected to this channel of input
    :param scale_factor: determines intensity of activation map  
    :param lines: Whether or not to draw horizontal and vertical lines separating input regions.
    :param figsize: Horizontal, vertical figure size in inches.
    :param cmap: Matplotlib colormap.
    :return: Used for re-drawing the weights plot.
    """

    n_sqrt = int(np.ceil(np.sqrt(lc.out_channels)))

    sel_slice = lc.w.view(lc.in_channels, lc.out_channels, lc.conv_size[0], lc.conv_size[1], lc.kernel_size[0], lc.kernel_size[1]).cpu()
    sel_slice = sel_slice[input_channel, ...]
    input_size = _pair(int(np.sqrt(lc.source.n)))
    reshaped = reshape_LC_weights(sel_slice, lc.out_channels, lc.kernel_size, lc.conv_size, input_size)
	
    ind_neuron = label * n_per_action + ind_neuron_in_group
    w = fc.w[:,ind_neuron].view(reshaped.shape[0]//lc.kernel_size[0],reshaped.shape[1]//lc.kernel_size[1])
    w = w.clip(lc.wmin,lc.wmax).repeat_interleave(lc.kernel_size[0], dim=0).repeat_interleave(lc.kernel_size[1], dim=1).cpu()

    fig, ax = plt.subplots(figsize=figsize)

    im = ax.imshow(reshaped.cpu()*w, cmap=cmap, vmin=lc.wmin, vmax=lc.wmax)
    div = make_axes_locatable(ax)
    cax = div.append_axes("right", size="5%", pad=0.05)

    if lines:
        for i in range(
            n_sqrt * lc.kernel_size[0],
            n_sqrt * lc.conv_size[0] * lc.kernel_size[0],
            n_sqrt * lc.kernel_size[0],
        ):
            ax.axhline(i - 0.5, color=color, linestyle="--")

        for i in range(
            n_sqrt * lc.kernel_size[1],
            n_sqrt * lc.conv_size[1] * lc.kernel_size[1],
            n_sqrt * lc.kernel_size[1],
        ):
            ax.axvline(i - 0.5, color=color, linestyle="--")

    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_aspect("auto")

    plt.colorbar(im, cax=cax)
    fig.tight_layout()

    return im

def plot_LC_activation_map(lc : object,
    spikes: torch.tensor,
    input_channel: int = 0,
    scale_factor: float = 1.0,
    lines: bool = True,
    figsize: Tuple[int, int] = (5, 5),
    cmap: str = "hot_r",
    color: str='r'
    ) -> AxesImage:
    # language=rst
    """
    Plot an activation map of a :code:`Connection` with `locally connected
    structure <http://yann.lecun.com/exdb/publis/pdf/gregor-nips-11.pdf>_.
    :param lc: LC connection object of LCNet
    :param input_channel: indicates weights which connected to this channel of input 
    :param scale_factor: determines intensity of activation map 
    :param lines: Whether or not to draw horizontal and vertical lines separating input regions.
    :param figsize: Horizontal, vertical figure size in inches.
    :param cmap: Matplotlib colormap.
    :return: Used for re-drawing the weights plot.
    """
    spikes = spikes.sum(0).squeeze().view(lc.conv_size[0]*int(np.sqrt(lc.out_channels)),lc.conv_size[1]*int(np.sqrt(lc.out_channels)))
    x = scale_factor * spikes / torch.max(spikes)
    x = x.clip(lc.wmin,lc.wmax).repeat_interleave(lc.kernel_size[0], dim=0).repeat_interleave(lc.kernel_size[1], dim=1).cpu()
    n_sqrt = int(np.ceil(np.sqrt(lc.out_channels)))

    sel_slice = lc.w.view(lc.in_channels, lc.out_channels, lc.conv_size[0], lc.conv_size[1], lc.kernel_size[0], lc.kernel_size[1]).cpu()
    sel_slice = sel_slice[input_channel, ...]
    input_size = _pair(int(np.sqrt(lc.source.n)))
    reshaped = reshape_LC_weights(sel_slice, lc.out_channels, lc.kernel_size, lc.conv_size, input_size)

    fig, ax = plt.subplots(figsize=figsize)

    im = ax.imshow(reshaped.cpu()*x, cmap=cmap, vmin=lc.wmin, vmax=lc.wmax)
    div = make_axes_locatable(ax)
    cax = div.append_axes("right", size="5%", pad=0.05)

    if lines:
        for i in range(
            n_sqrt * lc.kernel_size[0],
            n_sqrt * lc.conv_size[0] * lc.kernel_size[0],
            n_sqrt * lc.kernel_size[0],
        ):
            ax.axhline(i - 0.5, color=color, linestyle="--")

        for i in range(
            n_sqrt * lc.kernel_size[1],
            n_sqrt * lc.conv_size[1] * lc.kernel_size[1],
            n_sqrt * lc.kernel_size[1],
        ):
            ax.axvline(i - 0.5, color=color, linestyle="--")

    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_aspect("auto")

    plt.colorbar(im, cax=cax)
    fig.tight_layout()

    return im


def reshape_LC_weights(
    w: torch.Tensor,
    n_filters: int,
    kernel_size: Union[int, Tuple[int, int]],
    conv_size: Union[int, Tuple[int, int]],
    input_sqrt: Union[int, Tuple[int, int]],
) -> torch.Tensor:
    # language=rst
    """
    Get the weights from a locally connected layer and reshape them to be two-dimensional and square.
    :param w: Weights from a locally connected layer.
    :param n_filters: No. of neuron filters.
    :param kernel_size: Side length(s) of convolutional kernel.
    :param conv_size: Side length(s) of convolution population.
    :param input_sqrt: Sides length(s) of input neurons.
    :return: Locally connected weights reshaped as a collection of spatially ordered square grids.
    """
    k1, k2 = kernel_size
    c1, c2 = conv_size
    i1, i2 = input_sqrt
    c1sqrt, c2sqrt = int(math.ceil(math.sqrt(c1))), int(math.ceil(math.sqrt(c2)))
    fs = int(math.ceil(math.sqrt(n_filters)))

    w_ = torch.zeros((n_filters * k1, k2 * c1 * c2))

    for n1 in range(c1):
        for n2 in range(c2):
            for feature in range(n_filters):
                n = n1 * c2 + n2
                filter_ = w[feature, n1, n2, :, :
                ].view(k1, k2)
                w_[feature * k1 : (feature + 1) * k1, n * k2 : (n + 1) * k2] = filter_

    if c1 == 1 and c2 == 1:
        square = torch.zeros((i1 * fs, i2 * fs))

        for n in range(n_filters):
            square[
                (n // fs) * i1 : ((n // fs) + 1) * i2,
                (n % fs) * i2 : ((n % fs) + 1) * i2,
            ] = w_[n * i1 : (n + 1) * i2]

        return square
    else:
        square = torch.zeros((k1 * fs * c1, k2 * fs * c2))

        for n1 in range(c1):
            for n2 in range(c2):
                for f1 in range(fs):
                    for f2 in range(fs):
                        if f1 * fs + f2 < n_filters:
                            square[
                                k1 * (n1 * fs + f1) : k1 * (n1 * fs + f1 + 1),
                                k2 * (n2 * fs + f2) : k2 * (n2 * fs + f2 + 1),
                            ] = w_[
                                (f1 * fs + f2) * k1 : (f1 * fs + f2 + 1) * k1,
                                (n1 * c2 + n2) * k2 : (n1 * c2 + n2 + 1) * k2,
                            ]

        return square

def plot_semantic_pooling(lc : object,
    input_channel: int = 0,
    output_channel: int = None,
    lines: bool = True,
    figsize: Tuple[int, int] = (5, 5),
    cmap: str = "hot_r",
    color: str='r',
    ) -> AxesImage:
    # language=rst
    """
    Plot a connection weight matrix of a :code:`Connection` with `locally connected
    structure <http://yann.lecun.com/exdb/publis/pdf/gregor-nips-11.pdf>_.
    :param lc: LC connection object of LCNet
    :param input_channel: indicates weights which connected to this channel of input 
    :param output_channel: indicates weights of specific channel in the output layer
    :param lines: Whether or not to draw horizontal and vertical lines separating input regions.
    :param figsize: Horizontal, vertical figure size in inches.
    :param cmap: Matplotlib colormap.
    :return: Used for re-drawing the weights plot.
    """

    n_sqrt = int(np.ceil(np.sqrt(lc.out_channels)))
    sel_slice = lc.w.view(lc.in_channels, lc.out_channels, lc.conv_size[0], lc.conv_size[1], lc.kernel_size[0], lc.kernel_size[1]).cpu()
    input_size = _pair(int(np.sqrt(lc.source.n)))
    
    if output_channel is None:
        sel_slice = sel_slice[input_channel, ...]
        reshaped = reshape_LC_weights(sel_slice, lc.out_channels, lc.kernel_size, lc.conv_size, input_size)
    else:
        sel_slice = sel_slice[input_channel, output_channel, ...]
        sel_slice = sel_slice.unsqueeze(0)
        reshaped = reshape_LC_weights(sel_slice, 1, lc.kernel_size, lc.conv_size, input_size)
        print(reshaped.shape)

    fig, ax = plt.subplots(figsize=figsize)

    im = ax.imshow(reshaped.cpu(), cmap=cmap, vmin=lc.wmin, vmax=lc.wmax)
    div = make_axes_locatable(ax)
    cax = div.append_axes("right", size="5%", pad=0.05)

    if lines and  output_channel is None:
        for i in range(
            n_sqrt * lc.kernel_size[0],
            n_sqrt * lc.conv_size[0] * lc.kernel_size[0],
            n_sqrt * lc.kernel_size[0],
        ):
            ax.axhline(i - 0.5, color=color, linestyle="--")

        for i in range(
            n_sqrt * lc.kernel_size[1],
            n_sqrt * lc.conv_size[1] * lc.kernel_size[1],
            n_sqrt * lc.kernel_size[1],
        ):
            ax.axvline(i - 0.5, color=color, linestyle="--")

    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_aspect("auto")

    plt.colorbar(im, cax=cax)
    fig.tight_layout()

    return im

def plot_LC_weights(lc : object,
    input_channel: int = 0,
    output_channel: int = None,
    lines: bool = True,
    figsize: Tuple[int, int] = (5, 5),
    cmap: str = "hot_r",
    color: str='r',
    ) -> AxesImage:
    # language=rst
    """
    Plot a connection weight matrix of a :code:`Connection` with `locally connected
    structure <http://yann.lecun.com/exdb/publis/pdf/gregor-nips-11.pdf>_.
    :param lc: LC connection object of LCNet
    :param input_channel: indicates weights which connected to this channel of input 
    :param output_channel: indicates weights of specific channel in the output layer
    :param lines: Whether or not to draw horizontal and vertical lines separating input regions.
    :param figsize: Horizontal, vertical figure size in inches.
    :param cmap: Matplotlib colormap.
    :return: Used for re-drawing the weights plot.
    """

    n_sqrt = int(np.ceil(np.sqrt(lc.out_channels)))
    sel_slice = lc.w.view(lc.in_channels, lc.out_channels, lc.conv_size[0], lc.conv_size[1], lc.kernel_size[0], lc.kernel_size[1]).cpu()
    input_size = _pair(int(np.sqrt(lc.source.n)))
    
    if output_channel is None:
        sel_slice = sel_slice[input_channel, ...]
        reshaped = reshape_LC_weights(sel_slice, lc.out_channels, lc.kernel_size, lc.conv_size, input_size)
    else:
        sel_slice = sel_slice[input_channel, output_channel, ...]
        sel_slice = sel_slice.unsqueeze(0)
        reshaped = reshape_LC_weights(sel_slice, 1, lc.kernel_size, lc.conv_size, input_size)
        #print(reshaped.shape)

    fig, ax = plt.subplots(figsize=figsize)

    im = ax.imshow(reshaped.cpu(), cmap=cmap, vmin=lc.wmin, vmax=lc.wmax)
    div = make_axes_locatable(ax)
    cax = div.append_axes("right", size="5%", pad=0.05)

    if lines and  output_channel is None:
        for i in range(
            lc.kernel_size[0],#n_sqrt * lc.kernel_size[0],
            n_sqrt*lc.conv_size[0] * lc.kernel_size[0],#n_sqrt * lc.conv_size[0] * lc.kernel_size[0],
            lc.kernel_size[0],#,n_sqrt * lc.kernel_size[0],
        ):
            #print(i)
            ax.axhline(i, color=color, linestyle="--")

        for i in range(
            lc.kernel_size[1],#n_sqrt * lc.kernel_size[1],
            n_sqrt*lc.conv_size[1] * lc.kernel_size[1],#n_sqrt * lc.conv_size[1] * lc.kernel_size[1],
            lc.kernel_size[1],#n_sqrt * lc.kernel_size[1],
        ):
            ax.axvline(i, color=color, linestyle="--")
            
        for i in range(
            n_sqrt * lc.kernel_size[0],
            n_sqrt * lc.conv_size[0] * lc.kernel_size[0],
            n_sqrt * lc.kernel_size[0],
        ):
            #print(i)
            ax.axhline(i, color='b', linestyle="--")

        for i in range(
            n_sqrt * lc.kernel_size[1],
            n_sqrt * lc.conv_size[1] * lc.kernel_size[1],
            n_sqrt * lc.kernel_size[1],
        ):
            ax.axvline(i, color='b', linestyle="--")

    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_aspect("auto")

    plt.colorbar(im, cax=cax)
    fig.tight_layout()

    return im


# Design network

In [79]:
WANDB = True
if WANDB:
    !pip install -q wandb
    !wandb login
    import wandb

wandb: Currently logged in as: singularbrain (use `wandb login --relogin` to force relogin)


In [133]:
compute_size = lambda inp_size, k, s: int((inp_size-k)/s) + 1
def convergence(c):
    if c.norm is None:
        return 1-torch.mean((c.w-c.wmin)*(c.wmax-c.w))/((c.wmax-c.wmin)/2)**2
    else:
        mean_norm_factor = c.norm / c.w.shape[-1]
        return  1-(torch.mean((c.w-c.wmin)*(c.wmax-c.w))/((c.wmax-c.wmin)/2)**2)

        
class FCSNN(Network):
    def __init__(
        self,
        time,
        reward_fn,
        observation_period,
        decision_period,
        intensity,
        classic_reward,
        num_episodes,
        max_steps,
        nu,
        update_rule,
        inh_type,
        inh_factor,
        cartpole_online,
        save_path=None,
        load_path=None,
        dt=1,
        tc_trace=20,
        trace_additive=False,
        online=False,
        wandb_active = False,
        batch_size=1,
        save_interval = 100,
        wmin=0,
        wmax=1,
        clamp_intensity=None,
        **kwargs,
    ) -> None:
        # language=rst
        """
        Constructor for class ``FCSNN``.
        """
        manual_seed(SEED)
        super().__init__(dt=dt, reward_fn = None, online=online)
        self.cartpole_online = cartpole_online
        self.dt = dt
        self.intensity = intensity
        self.reward_fn = reward_fn
        self.batch_size = batch_size
        self.reward_fn.network = self
        self.reward_fn.dt = self.dt
        self.n_actions = 2
        self.n_classes = 2
        self.observation_period = observation_period,
        self.decision_period = decision_period,
        self.classic_reward = classic_reward
        self.neuron_per_class = 1
        self.neuron_per_action = 1
        self.save_path = save_path
        self.load_path = load_path
        self.save_interval = save_interval
        self.time = time
        self.clamp_intensity = clamp_intensity
        self.wmin = wmin 
        self.wmax = wmax
        self.wandb_active = wandb_active
        self.num_episodes = num_episodes
        self.max_steps= max_steps
        self.rew = 0.0
        self.local_rewarding = kwargs.get('local_rewarding', None)
        kwargs['target_name'] = None
        self.env = gym.make('CartPole-v0')
        self.env.reset()

      
        ### nodes
        inp = Input(shape= [2], traces=True, tc_trace=tc_trace, traces_additive = trace_additive)
        self.add_layer(inp, name="input")

        # main = LIFNodes(shape= [10], traces=True, tc_trace=tc_trace,
        #                      traces_additive = trace_additive)
        
        # self.add_layer(main, name="main")

        out = LIFNodes(shape= [2], traces=True, tc_trace=tc_trace,
                             traces_additive = trace_additive)

        self.add_layer(out, name="output")

        ### connections 
        inp_main = Connection(inp, out, nu = _pair(nu), update_rule = update_rule, wmin = wmin, wmax= wmax)

        #main_out = Connection(main, out, nu = _pair(nu), update_rule = update_rule, wmin = wmin, wmax= wmax)
       
        self.add_connection(inp_main, "input", "output")

        # self.add_connection(main_out, "main", "output")

        
        ### Inhibitory:
        if inh_type == 'between_layers':
            w = -inh_factor * torch.ones(out.n, out.n)
            for c in range(self.n_actions):
                w[c, c] = 0
            out_recurrent_inhibition = Connection(
                source=out,
                target=out,
                w=w,
                wmin=-inh_factor,
                wmax=0,
            )
            self.add_connection(out_recurrent_inhibition, "output", "output")            
        elif inh_type == None:
            pass

        # Directs network to GPU
        if gpu:
            self.to("cuda")


    def get_state_spiking(self,state):
        intensity = self.intensity

        s_norm = torch.tensor(([state[2]+self.env.theta_threshold_radians+0.05, state[3]*0.02+0.06])).type(torch.float32)
        f1 = lambda x: x * self.intensity
        a = f1(s_norm)
        f2 = lambda x: PoissonEncoder(time=self.time, dt=self.dt)(x)
        s_conv = f2(a)
        return s_conv

    def learn(
        self,
        hparams = None,
        online_validate = True,
        running_window_length = 250,
        verbose = True,
        **kwargs
    ):
        manual_seed(SEED)
        if self.wandb_active:
            wandb.watch(self)
        self.verbose = verbose

        reward_monitor = RewardMonitor(time =self.time)

        self.add_monitor(reward_monitor, name="reward")

        reward_hist = []

        self.spikes = {}
        for layer in set(self.layers):
            self.spikes[layer] = Monitor(self.layers[layer], state_vars=["s"], time=None)
            self.add_monitor(self.spikes[layer], name="%s_spikes" % layer)
            self.dopaminergic_layers = self.layers["output"]
       
        self.episode = 0
        rew = 0.0
        tot_rew = 0.0


        if self.load_path:
            
            self.model_params = torch.load(self.load_path)
            self.load_state_dict(torch.load(self.load_path)['state_dict'])
            self.episode =  self.model_params['episode']
            hparams = self.model_params['hparams']
            reward_hist = self.model_params['reward_hist']
            print(f'Previous model loaded! Resuming training from episode {self.episode}...\n')
        else:
            self.env = gym.make('CartPole-v0')
            self.env.reset()
            print(f'Previous model not found! Training from the beginning...\n')

        pbar = tqdm(total=self.num_episodes)
        
        kwargs['target_name'] = None
        for ep in range(self.num_episodes):
            # print(f"episode: {ep+1}")
            self.reset_state_variables()
            self.env.reset()
            done = False
            tot_rew = 0.0
            success = False
            failure = False
            num_steps = 0
            for t in count():
                
                if t != 0:
                    print("state (x, x_dot, theta (rad), theta_dot):", round(x,3), round(x_dot*0.02, 3), round(theta, 3), round(theta_dot*0.02,3), "output", sum_spikes, 'selected_action:',  selected_action, 'rew:', round(rew,3))
                    
                inp_spikes = self.get_state_spiking(self.env.state)
                if gpu:
                    inputs = {"input": inp_spikes.cuda().view(self.time, self.batch_size, 2)}
                else:
                    inputs = {"input": inp_spikes.view(self.time, self.batch_size, 2)}


                clamp = {}
                if self.clamp_intensity is not None:
                    encoder = PoissonEncoder(time = self.time, dt = self.dt)
                    clamp['output'] = encoder.enc(datum = torch.rand(self.layers['output'].n)*self.clamp_intensity,time = self.time, dt = self.dt)

                # if done:
                #     failure = True

                if t >= self.max_steps:
                    success = True
                    done = True
                elif done:
                    failure = True


                self.run(inputs=inputs, 
                        # time = self.time,
                        one_step=False,
                        clamp = clamp,
                        env = self.env,
                        success = success,
                        failure = failure,
                        **kwargs,
                        )
                
                rew = float(reward_monitor.get()[0])
                tot_rew += rew
                
                #main = self.spikes['main'].get('s')
                out_spikes = self.spikes["output"].get("s").view(self.time, self.batch_size, self.n_actions, self.neuron_per_action)
                sum_spikes = out_spikes.sum(0).sum(2)
                selected_action = torch.argmax(sum_spikes, dim=1)

                #self.spikes['main'].reset_state_variables()
                self.spikes["output"].reset_state_variables()
                reward_monitor.reset_state_variables()
                #self.reset_state_variables()

                if done:
                    if success == True:
                        print("output", sum_spikes, 'selected_action:',  selected_action, 'rew:', rew)
                        print('\Successful episode!')
                        num_steps = t+1
                    else:
                        print("output", sum_spikes, 'selected_action:',  selected_action, 'rew:', rew)
                        print('\nEpisode not successful!')
                        num_steps = t+1
                    break

                obs, reward, done, _ = self.env.step(int(selected_action[0]))     
                x = obs[0]
                x_dot = obs[1]
                theta = obs[2]
                theta_dot = obs[3]

                # Get voltage recording.
                #main_voltage = main_monitor.get("v")

                #tensorboard.update(step= i)


            

            if self.wandb_active:
                wandb.log({
                        **{'reward': tot_rew},
                        **{' to '.join(name) + ' std': c.w.std().item() for name, c in self.connections.items() if name[0]!=name[1]},
                        #**{name + ' spikes': monitor.get('s').sum().item() for name, monitor in self.spikes.items()},
                        **{' to '.join(name) + " gradients": wandb.Histogram(c.w.cpu()) for name, c in self.connections.items() if name[0]!=name[1]},
                    },
                    step = self.episode)


            #self.reward_fn.update() 
            #Plot_et.plot()    
            # self.reset_state_variables()  # Reset state variables.
            
            self.episode += 1
            reward_hist.append(tot_rew)
            print(f'\nEpisode {self.episode} lasted for {num_steps} time steps with total reward of {tot_rew}\n')
            if self.episode % self.save_interval == 0 and self.save_path:
                model_params = {'state_dict': self.state_dict(), 'hparams': network_hparams, 'episode': self.episode, 'reward_hist': reward_hist}
                torch.save(model_params, self.save_path)
                print("\n Model saved!\n")

            pbar.set_description_str("Episode: "+str(self.episode)+ ", Number of steps: " + str(num_steps) +", Episode Total Reward: " + "{:.2f}".format(tot_rew))
            pbar.update()

    
    def single_trial(self):
        self.reset_state_variables()
        
        image = self.get_state_spiking()

        if gpu:
            inputs = {"input": image.cuda().view(self.time, 1, self.in_channels, self.crop_size, self.crop_size)}
        else:
            inputs = {"input": image.view(self.time, 1, self.in_channels, self.crop_size, self.crop_size)}

        clamp = {}
        if self.clamp_intensity is not None:
            encoder = PoissonEncoder(time = self.time, dt = self.dt)
            clamp['output'] = encoder.enc(datum = torch.rand(self.layers['output'].n)*self.clamp_intensity,time = self.time, dt = self.dt)

        self.run(inputs=inputs, 
                time=self.time, 
                one_step = False,
                mode = 'RL',
                env = 'CartPole'
                )
    

# Set up hyper-parameters

In [146]:
WANDB = True

network_hparams = {
    'time': 100,
    'reward_fn': RLTasks('CartPole-v0'),
    'observation_period': 1,
    'decision_period': 1,
    'intensity': 400,
    'classic_reward': True,
    'num_episodes': 10000,
    'max_steps': 200,
    'nu': 0.01,
    'update_rule': MSTDPET,
    'inh_type': None,
    'inh_factor': 100,
    'cartpole_online': True,
    'save_path': '/content/drive/My Drive/LCNet/FCSNN_cartpole.pth',
    'load_path': None,#'/content/drive/My Drive/LCNet/FCSNN_cartpole.pth',
    'dt':1,
    'tc_trace':20,
    'trace_additive':False,
    'online':False,
    'wandb_active':WANDB,
    'batch_size':1,
    'save_interval':100,
    'wmin':0,
    'wmax':1,
    'clamp_intensity':None,
}

# Learning

In [147]:
if WANDB:
    wandb.init(project='FCSNN_CartPole', entity='singularbrain', config=network_hparams)

input to output std,▁▂▅▆████████████▇███████████████████████
reward,▆▆▃▃█▁▆▆▆▃▆▃▆▆▆▆▆▃▃▆▆▃▁▃▆▆▆█▁▃█▃▆▆▆▃▁▃▆▆
input to output std,0.4185
reward,10.0


In [148]:
from google.colab import drive
if network_hparams['save_path'] or network_hparams['load_path']:    
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
manual_seed(SEED)


net = FCSNN(**network_hparams)
net.learn(**network_hparams)

Previous model not found! Training from the beginning...



  0%|          | 0/10000 [00:00<?, ?it/s]

state (x, x_dot, theta (rad), theta_dot): -0.018 -0.004 -0.024 0.005 output tensor([[0, 0]]) selected_action: tensor([0]) rew: 1.0
state (x, x_dot, theta (rad), theta_dot): -0.022 -0.008 -0.018 0.011 output tensor([[0, 0]]) selected_action: tensor([0]) rew: 1.0
state (x, x_dot, theta (rad), theta_dot): -0.03 -0.012 -0.007 0.017 output tensor([[0, 0]]) selected_action: tensor([0]) rew: 1.0
state (x, x_dot, theta (rad), theta_dot): -0.041 -0.016 0.01 0.023 output tensor([[0, 0]]) selected_action: tensor([0]) rew: 1.0
state (x, x_dot, theta (rad), theta_dot): -0.057 -0.019 0.032 0.029 output tensor([[0, 0]]) selected_action: tensor([0]) rew: 1.0
state (x, x_dot, theta (rad), theta_dot): -0.076 -0.023 0.061 0.035 output tensor([[0, 0]]) selected_action: tensor([0]) rew: 1.0
state (x, x_dot, theta (rad), theta_dot): -0.1 -0.027 0.096 0.041 output tensor([[0, 0]]) selected_action: tensor([0]) rew: 1.0
state (x, x_dot, theta (rad), theta_dot): -0.127 -0.031 0.137 0.047 output tensor([[0, 0]])